In [ ]:
%matplotlib widget

In [ ]:
!pip install --upgrade diffusers[torch]
!pip install transformers

In [ ]:
from bayes3d.neural.dino import Dino
import bayes3d as b
import os
import torch
import numpy as np
import matplotlib
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import torchvision.transforms as T
import bayes3d.utils.ycb_loader
from bayes3d.viz.open3dviz import Open3DVisualizer
from tqdm import tqdm
import open3d as o3d


In [ ]:
b.setup_visualizer()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dinov2_vitg14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
dino = dinov2_vitg14.to(device)  # Same issue with larger model

In [ ]:
def get_embeddings(dinov2_vitg14, rgb):
    img = b.get_rgb_image(rgb).convert('RGB')
    patch_w, patch_h = np.array(img.size) // 14
    transform = T.Compose([
        T.GaussianBlur(9, sigma=(0.1, 2.0)),
        T.Resize((patch_h * 14, patch_w * 14)),
        T.CenterCrop((patch_h * 14, patch_w * 14)),
        T.ToTensor(),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])
    tensor = transform(img)[:3].unsqueeze(0).to(device)
    with torch.no_grad():
        features_dict = dinov2_vitg14.forward_features(tensor)
        features = features_dict['x_norm_patchtokens'][0].reshape((patch_h, patch_w, 384)).permute(2, 0, 1).unsqueeze(0)
    img_feat_norm = torch.nn.functional.normalize(features, dim=1)
    output = jnp.array(img_feat_norm.cpu().detach().numpy())[0]
    del img_feat_norm
    del features
    del tensor
    del features_dict
    torch.cuda.empty_cache()
    return jnp.transpose(output, (1,2,0))

In [ ]:
w,h = 1400,1400
intrinsics = b.Intrinsics(
    height=h,
    width=w,
    fx=2000.0, fy=2000.0,
    cx=w/2.0, cy=h/2.0,
    near=0.001, far=6.0
)
scaled_down_intrinsics = b.camera.scale_camera_parameters(intrinsics, 1.0/14.0)
scaled_down_intrinsics

In [ ]:
b.setup_renderer(scaled_down_intrinsics)

In [ ]:
model_dir = os.path.join(b.utils.get_assets_dir(),"bop/ycbv/models")
mesh_paths = []
for idx in range(1,22):
    mesh_paths.append(os.path.join(model_dir,"obj_" + "{}".format(idx).rjust(6, '0') + ".ply"))
SCALING_FACTOR = 1.0/1000.0


In [ ]:
gaussian_vmf_parallel = jax.vmap(b.distributions.gaussian_vmf, (0, None, None, None))
split_jit = jax.jit(jax.random.split, static_argnums=(1,))

In [ ]:
viz = Open3DVisualizer(intrinsics)


In [ ]:
viz.clear()
obj_idx = 2
mesh_filename = mesh_paths[obj_idx]
mesh = o3d.io.read_triangle_model(mesh_filename)
mesh.meshes[0].mesh.scale(SCALING_FACTOR, np.array([0.0, 0.0, 0.0]))
viz.render.scene.add_model(f"1", mesh)

In [ ]:
object_poses = jnp.array([b.t3d.inverse_pose(b.t3d.transform_from_pos_target_up(
            jnp.array([0.0, 0.6, 0.0]),
            jnp.array([0.0, 0.0, 0.0]),
            jnp.array([0.0, 0.0, 1.0]),
        )) @ b.t3d.transform_from_axis_angle(jnp.array([0.0, 0.0, 1.0]), angle)  for angle in np.linspace(-jnp.pi, jnp.pi, 101)[:-1]])
# for (i, pose) in enumerate(object_poses):
#     b.show_pose(f"{i}", pose)

In [ ]:
images = []
for i, pose in tqdm(enumerate(object_poses)):
    # if i > 0:
    #     mesh.meshes[0].mesh.transform(b.inv
    
    # viz.render.scene.add_model(f"1", mesh)
    rgbd = viz.capture_image(intrinsics, b.t3d.inverse_pose(pose))
    images.append(rgbd)

# jnp.savez(f"data_{obj_idx}.npz",images=images, camera_poses=camera_poses)

In [ ]:
b.get_rgb_image(images[0].rgb)

In [ ]:
def get_heatmap(target_embedding, all_embeddings):
    dot_products = jnp.einsum("i, abi->ab", target_embedding, all_embeddings)
    heatmap = dot_products / 2.0 + 0.5
    return heatmap
# heatmap = get_heatmap(img1_embedding[45,45], img2_embedding)
# scaled_up_heatmap = b.utils.resize(heatmap, img2.shape[0], img2.shape[1])
# plt.matshow(heatmap)

In [ ]:
bop_ycb_dir = os.path.join(b.utils.get_assets_dir(), "bop/ycbv")
rgbd, gt_ids, gt_poses, masks = b.utils.ycb_loader.get_test_img('51', '1', bop_ycb_dir)



img2 = rgbd.scale_rgbd(3.0).rgb
img2_embedding = get_embeddings(dinov2_vitg14, img2)

b.get_rgb_image(img2)


In [ ]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
data = []


IDX1 = 70
img1 = images[IDX1].rgb
img1_embedding = get_embeddings(dinov2_vitg14, img1)
img1_resized = images[IDX1].scale_rgbd(1.0/14.0).rgb

axes[0].imshow(img1 /255.0)
axes[1].imshow(img2 / 255.0)
def onclick(event):
    print("hello")
    # cos = nn.CosineSimilarity(dim=1)

    x, y = int(np.round(event.xdata)), int(np.round(event.ydata))
    x2, y2 = int(np.round(event.xdata / 14.0)), int(np.round(event.ydata / 14.0))
    data.append((x,y))

    target_embedding = img1_embedding[y2, x2,:]

    # dot_products = jnp.einsum("i, abi->ab", target_embedding, img2_embedding)

    heatmap = get_heatmap(target_embedding, img2_embedding)
    scaled_up_heatmap = b.utils.resize(heatmap, img2.shape[0], img2.shape[1])

    axes[0].clear()
    axes[1].clear()
    axes[0].imshow(img1 /255.0)
    axes[1].imshow(img2 / 255.0)
    axes[0].scatter(x, y, c='r', s=10.0)

    max_yx = np.unravel_index(scaled_up_heatmap.argmax(), scaled_up_heatmap.shape)
    # axes[1].axis('off')
    axes[1].imshow(255 * scaled_up_heatmap, alpha=0.45, cmap='viridis')
    # axes[1].axis('off')
    axes[1].scatter(max_yx[1], max_yx[0], c='r', s=10)
    # axes[1].set_title('target image')
    # gc.collect()

fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()

In [ ]:
print(data)

In [ ]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, 2 * np.pi)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line, = ax.plot(x, np.sin(x))

def update(w = 1.0):
    line.set_ydata(np.sin(w * x))
    fig.canvas.draw_idle()

interact(update);

In [ ]:
# data = jnp.load(f"data_{obj_idx}.npz", allow_pickle=True)
# images = data["images"]
# camera_poses = data["camera_poses"]

# Training

In [ ]:
num_images = len(images)
num_training_images = 10
training_indices = jnp.arange(0,num_images-1, num_images // num_training_images)
# b.hstack_images([
#     b.get_rgb_image(images[idx].rgb) for idx in training_indices
# ])

In [ ]:
keypoint_coordinates = []
keypoint_embeddings = []

key = jax.random.PRNGKey(10)
sparse_descriptors = []
for iteration in range(len(training_indices)):
    index = training_indices[iteration]
    index_next = training_indices[(iteration+1) % len(training_indices)]
    print(index, index_next)
    keys = jax.random.split(key)[1]
    
    training_image = images[index]
    object_pose = object_poses[index]
    
    scaled_down_training_image = training_image.scale_rgbd(1.0/14.0)
    embedding_image = get_embeddings(training_image)
    embedding_image_next = get_embeddings(images[index_next])
    
    foreground_mask = (jnp.inf != scaled_down_training_image.depth)
    foreground_pixel_coordinates = jnp.transpose(jnp.vstack(jnp.where(foreground_mask)))
    
    depth = jnp.array(scaled_down_training_image.depth)
    depth = depth.at[depth == jnp.inf].set(0.0)
    point_cloud_image = b.t3d.unproject_depth(depth, scaled_down_training_image.intrinsics)
    point_cloud_image_object_frame = b.t3d.apply_transform(point_cloud_image, b.t3d.inverse_pose(object_pose))
    
    scaled_down_training_image_next = images[index_next].scale_rgbd(1.0/14.0)
    depth = jnp.array(scaled_down_training_image_next.depth)
    depth = depth.at[depth == jnp.inf].set(0.0)
    point_cloud_image_next = b.t3d.unproject_depth(depth, scaled_down_training_image_next.intrinsics)
    point_cloud_image_next_object_frame = b.t3d.apply_transform(point_cloud_image_next, b.t3d.inverse_pose(object_poses[index_next]))
    
    embeddings_subset = embedding_image[foreground_pixel_coordinates[:,0], foreground_pixel_coordinates[:,1],:]
    coordinates_subset = point_cloud_image_object_frame[foreground_pixel_coordinates[:,0], foreground_pixel_coordinates[:,1],:]
    similarity_embedding = jnp.einsum("abi, ki->abk", embedding_image_next, embeddings_subset)
    best_match = similarity_embedding.argmax(-1)
    distance_to_best_match = jnp.linalg.norm(point_cloud_image_next_object_frame - coordinates_subset[best_match,:], axis=-1)
    
    selected = (distance_to_best_match < 0.01) * (similarity_embedding.max(-1) > 0.9)
    subset = jnp.unique(best_match[selected])


    _keypoint_embeddings = embedding_image[foreground_pixel_coordinates[subset,0], foreground_pixel_coordinates[subset,1],:]
    keypoint_world_coordinates = point_cloud_image[foreground_pixel_coordinates[subset,0], foreground_pixel_coordinates[subset,1],:]
    _keypoint_coordinates = b.t3d.apply_transform(keypoint_world_coordinates, b.t3d.inverse_pose(object_pose))

    keypoint_coordinates.append(_keypoint_coordinates)
    keypoint_embeddings.append(_keypoint_embeddings)
    del embedding_image
    del embedding_image_next

keypoint_coordinates = jnp.concatenate(keypoint_coordinates)
keypoint_embeddings = jnp.concatenate(keypoint_embeddings)
print(keypoint_coordinates.shape)
print(keypoint_embeddings.shape)

In [ ]:
# keypoint_coordinates = []
# keypoint_embeddings = []

# key = jax.random.PRNGKey(10)
# sparse_descriptors = []

# for iteration in range(len(training_indices)):
#     index = training_indices[iteration]
#     index_next = training_indices[(iteration+1) % len(training_indices)]
#     print(index, index_next)
#     keys = jax.random.split(key)[1]
    
#     training_image = images[index]
#     object_pose = object_poses[index]
    
#     scaled_down_training_image = training_image.scale_rgbd(1.0/14.0)
#     embedding_image = get_embeddings(training_image)

#     # del embeddings
#     foreground_mask = (jnp.inf != scaled_down_training_image.depth)
#     foreground_pixel_coordinates = jnp.transpose(jnp.vstack(jnp.where(foreground_mask)))

#     NUM_KEYPOINTS_TO_SELECT = jnp.min(jnp.array([2000,foreground_pixel_coordinates.shape[0]]))
#     subset = jax.random.choice(jax.random.PRNGKey(10),foreground_pixel_coordinates.shape[0], shape=(NUM_KEYPOINTS_TO_SELECT,), replace=False)

#     depth = jnp.array(scaled_down_training_image.depth)
#     depth = depth.at[depth == jnp.inf].set(0.0)
#     point_cloud_image = b.t3d.unproject_depth(depth, scaled_down_training_image.intrinsics)

#     keypoint_world_coordinates = point_cloud_image[foreground_pixel_coordinates[subset,0], foreground_pixel_coordinates[subset,1],:]
#     _keypoint_coordinates = b.t3d.apply_transform(keypoint_world_coordinates, b.t3d.inverse_pose(object_pose))
#     _keypoint_embeddings = embedding_image[foreground_pixel_coordinates[subset,0], foreground_pixel_coordinates[subset,1],:]
    
#     keypoint_coordinates.append(_keypoint_coordinates)
#     keypoint_embeddings.append(_keypoint_embeddings)
#     del embedding_image
# keypoint_coordinates = jnp.concatenate(keypoint_coordinates)
# keypoint_embeddings = jnp.concatenate(keypoint_embeddings)
# print(keypoint_coordinates.shape)
# print(keypoint_embeddings.shape)

In [ ]:

def render_embedding_image(pose, keypoint_coordinates, keypoint_embeddings):
    point_cloud_img = b.RENDERER.render(pose[None,...], jnp.array([0]))[:,:,:3]
    point_cloud_img_in_object_frame = b.t3d.apply_transform(point_cloud_img, b.t3d.inverse_pose(pose))

    distances_to_keypoints = (
        jnp.linalg.norm(point_cloud_img_in_object_frame[:, :,None,...] - keypoint_coordinates[None, None,:,...],
        axis=-1
    ))
    index_of_nearest_keypoint = distances_to_keypoints.argmin(2)
    distance_to_nearest_keypoints = distances_to_keypoints.min(2)

    DISTANCE_THRESHOLD = 0.04
    valid_match_mask = (distance_to_nearest_keypoints < DISTANCE_THRESHOLD)[...,None]
    selected_keypoints = keypoint_coordinates[index_of_nearest_keypoint]
    rendered_embeddings_image = keypoint_embeddings[index_of_nearest_keypoint] * valid_match_mask
    return point_cloud_img, rendered_embeddings_image

vmf_score = lambda q, q_mean, conc: tfp.distributions.VonMisesFisher(
    q_mean, conc
).log_prob(q)

import functools
from functools import partial
from tensorflow_probability.substrates import jax as tfp


@functools.partial(
    jnp.vectorize,
    signature='(m),(m)->()',
    excluded=(2,),
)
def vmf_vectorize(
    embeddings,
    embeddings_mean,
    conc
):
    return vmf_score(embeddings, embeddings_mean, conc)


def score_pose(pose, keypoint_coordinates, keypoint_embeddings, observed_embeddings):
    _,rendered_embedding_image = render_embedding_image(pose, keypoint_coordinates, keypoint_embeddings)
    scores = vmf_vectorize(observed_embeddings, rendered_embedding_image, 1000.0)
    return scores

def get_pca(embeddings):
    features_flat = torch.from_numpy(np.array(embeddings).reshape(-1, embeddings.shape[-1]))
    U, S, V = torch.pca_lowrank(features_flat - features_flat.mean(0), niter=10)
    proj_PCA = jnp.array(V[:, :3])
    return proj_PCA

def get_colors(features, proj_V):
    features_flat = features.reshape(-1, features.shape[-1])
    feat_rgb = features_flat @ proj_V
    feat_rgb = (feat_rgb + 1.0) / 2.0
    feat_rgb = feat_rgb.reshape(features.shape[:-1] + (3,))
    return feat_rgb

score_pose_jit = jax.jit(score_pose)
score_pose_parallel_jit = jax.jit(jax.vmap(score_pose, in_axes=(0, None, None, None )))

In [ ]:
import meshcat.geometry as g

proj_V = get_pca(keypoint_embeddings)
colors = get_colors(keypoint_embeddings, proj_V)
b.clear()
obj = g.PointCloud(np.transpose(keypoint_coordinates)*10.0, np.transpose(colors), size=0.1)
b.meshcatviz.VISUALIZER["2"].set_object(obj)

In [ ]:
b.setup_renderer(scaled_down_training_image.intrinsics)
b.RENDERER.add_mesh_from_file(mesh_filename, scaling_factor=SCALING_FACTOR)
b.setup_renderer(scaled_down_training_image.intrinsics)
b.RENDERER.add_mesh_from_file(mesh_filename, scaling_factor=SCALING_FACTOR)

In [ ]:
b.RENDERER.render(jnp.eye(4)[None,...], jnp.array([0]));
pc_img, rendered_embedding_image = render_embedding_image(object_poses[0], keypoint_coordinates, keypoint_embeddings);

In [ ]:
IDX = 15
test_rgbd = images[IDX]
test_rgbd_scaled = test_rgbd.scale_rgbd(1.0/14.0)
observed_embeddings = get_embeddings(test_rgbd)
# b.get_rgb_image(test_rgbd.rgb)

In [ ]:
posterior = jnp.concatenate([score_pose_parallel_jit(i,  keypoint_coordinates, keypoint_embeddings, observed_embeddings)[:,test_rgbd_scaled.depth != jnp.inf].mean(-1) for i in jnp.array_split(object_poses, 10)])
print(posterior.argmax())
best_pose = object_poses[posterior.argmax()]
print(best_pose)

In [ ]:
# predicted_poses = []
# for IDX in tqdm(range(len(images))):
#     test_rgbd = images[IDX]
#     test_rgbd_scaled = test_rgbd.scale_rgbd(1.0/14.0)
#     observed_embeddings = get_embeddings(test_rgbd)
#     posterior = jnp.concatenate([score_pose_parallel_jit(i,  keypoint_coordinates, keypoint_embeddings, observed_embeddings)[:,test_rgbd_scaled.depth != jnp.inf].mean(-1) for i in jnp.array_split(object_poses, 10)])
#     print(posterior.argmax())
#     best_pose = object_poses[posterior.argmax()]
#     print(best_pose)
#     predicted_poses.append(best_pose)
#     del observed_embeddings

In [ ]:
observed_embedding_colors = get_colors(observed_embeddings, proj_V)
observed_embeddings_image_viz = b.get_rgb_image(observed_embedding_colors * 255.0)

pc_img, rendered_embedding_image = render_embedding_image(best_pose, keypoint_coordinates, keypoint_embeddings)
colors = get_colors(rendered_embedding_image, proj_V)
rgba = jnp.array(b.get_rgb_image(colors * 255.0))
# rgba = rgba.at[pc_img[:,:,2] > intrinsics.far - 0.01, :3].set(255.0)
rerendered_embeddings_viz = b.get_rgb_image(rgba)

b.multi_panel([
    b.get_rgb_image(test_rgbd.rgb), 
    b.scale_image(observed_embeddings_image_viz, 14.0),
    b.scale_image(rerendered_embeddings_viz, 14.0)
],labels=[
    "Observed RGB",
    "Embeddings",
    "Reconstruction"
],label_fontsize=50
).convert("RGB")

In [ ]:
random_pose = b.transform_from_pos(jnp.array([0.0, 0.0, 0.6])) @ b.distributions.vmf_jit(jax.random.PRNGKey(40), 0.001)
test_rgbd = viz.capture_image(intrinsics, b.t3d.inverse_pose(random_pose))
test_rgbd_scaled = test_rgbd.scale_rgbd(1.0/14.0)
observed_embeddings = get_embeddings(test_rgbd)
b.get_rgb_image(test_rgbd.scale_rgbd(0.2).rgb)

In [ ]:
observed_embedding_colors = get_colors(observed_embeddings, proj_V)
observed_embeddings_image_viz = b.get_rgb_image(observed_embedding_colors * 255.0)
observed_embeddings_image_viz

In [ ]:
match_scores = jnp.einsum("abk,ck",observed_embeddings, keypoint_embeddings)
top_match = match_scores.max(-1)
top_match_idx = match_scores.argmax(-1)

THRESHOLD = 0.8
match_mask = (top_match > THRESHOLD) * (test_rgbd_scaled.depth < test_rgbd_scaled.intrinsics.far)
print(match_mask.sum())
b.get_depth_image(1.0 * match_mask)

In [ ]:
observed_point_cloud_image = b.unproject_depth_jit(test_rgbd_scaled.depth, test_rgbd_scaled.intrinsics)

observed_match_coordinates = observed_point_cloud_image[match_mask,:]
model_coordinates = keypoint_coordinates[top_match_idx[match_mask],:]

b.clear()
b.show_cloud("1", observed_match_coordinates.reshape(-1,3))
b.show_cloud("2", model_coordinates.reshape(-1,3), color=b.RED)

In [ ]:
b.clear()
estimated_pose = b.estimate_transform_between_clouds(model_coordinates, observed_match_coordinates)
estimated_pose = b.distributions.gaussian_vmf_jit(keys[10],random_pose, 0.1, 10.0)
b.show_trimesh("mesh", b.RENDERER.meshes[0])
b.set_pose("mesh", estimated_pose)
b.show_cloud("obs", observed_point_cloud_image.reshape(-1,3))


In [ ]:
print(score_pose(random_pose, keypoint_coordinates, keypoint_embeddings, observed_embeddings)[test_rgbd_scaled.depth != jnp.inf].mean(-1))
print(score_pose(estimated_pose, keypoint_coordinates, keypoint_embeddings, observed_embeddings)[test_rgbd_scaled.depth != jnp.inf].mean(-1))

In [ ]:
keys = split_jit(jax.random.PRNGKey(10), 100)

In [ ]:
for _ in range(20):
    potential_poses = gaussian_vmf_parallel(keys,estimated_pose, 0.01, 20000.0)
    current_score = score_pose(estimated_pose, keypoint_coordinates, keypoint_embeddings, observed_embeddings)[test_rgbd_scaled.depth != jnp.inf].mean(-1)
    scores = score_pose_parallel_jit(potential_poses,  keypoint_coordinates, keypoint_embeddings, observed_embeddings)[:,test_rgbd_scaled.depth != jnp.inf].mean(-1)
    if scores.max() > current_score:
        estimated_pose = potential_poses[scores.argmax()]
    keys = split_jit(keys[0], 100)
    print(scores.max(), current_score)
    b.show_trimesh("mesh", b.RENDERER.meshes[0])
    b.set_pose("mesh", estimated_pose)
    b.show_cloud("obs", observed_point_cloud_image.reshape(-1,3))


In [ ]:
print(score_pose(random_pose, keypoint_coordinates, keypoint_embeddings, observed_embeddings)[test_rgbd_scaled.depth != jnp.inf].mean(-1))
print(score_pose(estimated_pose, keypoint_coordinates, keypoint_embeddings, observed_embeddings)[test_rgbd_scaled.depth != jnp.inf].mean(-1))

# 

In [ ]:
observed_embedding_colors = get_colors(observed_embeddings, proj_V)
observed_embeddings_image_viz = b.get_rgb_image(observed_embedding_colors * 255.0)

pc_img, rendered_embedding_image = render_embedding_image(estimated_pose, keypoint_coordinates, keypoint_embeddings)
colors = get_colors(rendered_embedding_image, proj_V)
rgba = jnp.array(b.get_rgb_image(colors * 255.0))
# rgba = rgba.at[pc_img[:,:,2] > intrinsics.far - 0.01, :3].set(255.0)
rerendered_embeddings_viz = b.get_rgb_image(rgba)

b.multi_panel([
    b.get_rgb_image(test_rgbd.rgb), 
    b.scale_image(observed_embeddings_image_viz, 14.0),
    b.scale_image(rerendered_embeddings_viz, 14.0)
],labels=[
    "Observed RGB",
    "Embeddings",
    "Reconstruction"
],label_fontsize=50
).convert("RGB")

In [ ]:
# b.clear()
# b.show_trimesh("mesh", b.RENDERER.meshes[obj_idx])
b.show_cloud("obs", observed_point_cloud_image.reshape(-1,3))
